#### Guideline

![](example_guideline.jpg)

#### Reference  

~~~

### hidden layer까지 있는 경우  
- adjacency matrix의 일종인 mat_mask를 통해서 hidden layer들의 dimension 정보를 담고 있는 hidden_dim_list를 산출  

__1. Architecture을 짜는 코드는 다음과 같이 세 파트__   

- input layer와 첫번째 hidden layer을 이어줌(connection별로)
- hidden layer와 hidden layer들끼리 이어줌(반복문으로 가야함, 총 # of hidden layer - 1 만큼 반복)
- 마지막 hidden layer와 output layer를 이어줌 

__2. layer와 layer을 이어줄 때__   

- 나중 layer의 노드들 기준으로 반복이 이루어져야함(forward시에는 value가 나중 layer의 노드로 합쳐지면서 흘러가니까)   
- 각 connection에 대해서 activation function이 다르기 때문에 node 기준으로 architecture을 짜 주어야 함 -> node dictionary를 쓸건데, __backprop이 가능하게 하기 위해서는 어떤 자료형?을 써야 하는지 고민해야__  
    - node = {}
    - node\['hidden1', ~~~\]

__3. 추가 고려사항__   

- skip connection 사용가능할건지, 사용한다면 구현상으로 어떻게 달라질지   

In [2]:
import numpy as np
import torch
from torch import nn

In [3]:
# mat_mask 행렬 바로 생성 
mat_mask = np.array([[0,2,0,0,2,0,0,0,0,0],[2,0,2,0,0,0,0,0,0,0],[0,2,0,2,0,0,0,0,0,0],[0,0,0,0,0,1,1,0,0,0],[0,0,0,0,0,0,1,1,0,0],[0,0,0,0,0,0,0,0,0,3],[0,0,0,0,0,0,0,0,3,0]])
mat_mask = torch.from_numpy(mat_mask)
print(mat_mask)
print(mat_mask.shape) 

tensor([[0, 2, 0, 0, 2, 0, 0, 0, 0, 0],
        [2, 0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 0, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
        [0, 0, 0, 0, 0, 0, 0, 0, 3, 0]], dtype=torch.int32)
torch.Size([7, 10])


In [ ]:
# hidden layer 추가한 버전
#아래 코드를 좀 변형하면 될듯? 아래 코드는 hidden layer가 없어서.. 
class MAT_to_WANN(nn.Module):
    def __init__(self, in_dim, out_dim, mat_mask):
        """
       :param in_features: number of input features
       :param out_features: number of output features
       :param indices_mask: list of two lists containing indices for dimensions 0 and 1, used to create the mask  
       """
        super(MaskedLinear, self).__init__()    
        
        def calculate_hidden_dim(in_dim, out_dim, mat_mask):
            #hidden dim의 정보를 담은 list를 출력한다..?
            hidden_dim_list = []
            start_col_idx = 0 
            finish_col_idx = in_dim - 1
            
            while(True):
                if finish_col_idx >= mat_mask.shape[1]:
                    break 
        
                for i in range(start_col_idx, len(mat_mask)): 
                    if (mat_mask[i,start_col_idx:(finish_col_idx + 1)].sum() == 0):
                        hidden_dim = i - sum(hidden_dim_list)
                        hidden_dim_list += [hidden_dim]  
                        start_col_idx = finish_col_idx + 1
                        finish_col_idx += i 
                        break 
                        
            return hidden_dim_list
        
    
    
    #이부분 바꿔야함    
    def forward(self, input):   
        
        #각 layer별로, 그 안에서 노드별로 전 layer에서 값들을 받아서 더해져야함
        #아래는 아직 수도코드임 
        #첫번째 hidden layer의 노드별로 선언  
        hidden1_node1_value = nn.ReLU(nn.Linear(1,1)(input_node2_value)) + nn.ReLU(nn.Linear(1,1)(input_node4_value))  
        hidden1_node2_value = nn.ReLU(nn.Linear(1,1)(input_node3_value))  
        hidden1_node3_value = nn.ReLU(nn.Linear(1,1)(input_node1)) + nn.ReLU(nn.Linear(1,1)(input_node4))  
        
        #두번째 hidden layer의 노드별로 선언
        hidden2_node1_value = nn.Linear(1,1)(hidden1_node1_value) + nn.Linear(1,1)(hidden1_node2_value) 
        hidden2_node2_value = nn.Linear(1,1)(hidden1_node2_value) + nn.Linear(1,1)(hidden1_note3_value)  
        
        #output layer의 노드별로 선언
        output_node1_value = nn.Sigmoid(nn.Linear(1,1)(hidden2_node2_value))
        output_node2_value = nn.Sigmoid(nn.Linear(1,1)(hidden2_node1_value)) 
        
        out = torch.concat([output_node1_value, output_node2_value])  
        
        
        return out

    
    
    
    
